In [1]:
import sys; print(sys.version)
import os
import glob
import subprocess
import multiprocessing
import io
from collections import OrderedDict
import json

import numpy as np; print('numpy', np.__version__)
import pandas as pd; print('pandas',pd.__version__)
import allel; print('scikit-allel', allel.__version__)
import zarr; print('zarr', zarr.__version__)
import scipy.stats

import matplotlib as mpl

import statsmodels; print('statsmodels', statsmodels.__version__)
import statsmodels.api as sm

from IPython.display import display, HTML

3.6.7 | packaged by conda-forge | (default, Feb 28 2019, 09:07:38) 
[GCC 7.3.0]
numpy 1.16.2
pandas 0.24.1
scikit-allel 1.2.0
zarr 2.2.0
statsmodels 0.9.0


In [2]:
%matplotlib notebook
mpl.rcParams['figure.facecolor'] = '#BBBBBB'

## Parameters

In [3]:
## Uncommnet blocks for each different set of samples

# SETNAME = 'VGL-gam'
# GENOME = 'AgamP4.11'
# ZVCF = 'vcfs/YL-Agam-GF2_pflit.vcf.gz.zarr'
# SAMPLE_LIST_FN = 'vcfs/hanno_Agam_sample_list.txt'

# SETNAME = 'Ag1000g-gam'
# GENOME = "AgamP4.11"
# ZVCF = '/data/ag1000g_p2_ar1/ngs.sanger.ac.uk/production/ag1000g/phase2/AR1/variation/main/vcf/all/ag1000g.phase2.ar1.zarr'
# SAMPLE_LIST_FN = ['S', '/data/ag1000g_p2_ar1/samples/samples.meta.txt'] # list implies ag1000g to be filtered on m_s

# SETNAME = 'VGL-col'
# GENOME = "AgamP4.11"
# ZVCF = "vcfs/100Acol_pflit.vcf.gz.zarr"
# SAMPLE_LIST_FN = None # use all samples in ZVCF

# SETNAME = 'Ag1000g-col'
# GENOME = "AgamP4.11"
# ZVCF = '/data/ag1000g_p2_ar1/ngs.sanger.ac.uk/production/ag1000g/phase2/AR1/variation/main/vcf/all/ag1000g.phase2.ar1.zarr'
# SAMPLE_LIST_FN = ['M', '/data/ag1000g_p2_ar1/samples/samples.meta.txt'] # list implies ag1000g to be filtered on m_s

SETNAME = 'VGL-Aaeg'
GENOME = "Aaegypti_L5.1"
ZVCF = "vcfs/YL-Aaeg-03_pflit.vcf.gz.zarr"
SAMPLE_LIST_FN = None # use all samples in ZVCF

## automatically set based on GENOME
if GENOME == 'AgamP4.11':
    GENE_PREFIX = "AGAP"
    CHROMS = ['2R','2L','3R','3L','X']
    REF_FAI_FN = '/data/reference/Anopheles-gambiae-PEST_CHROMOSOMES_AgamP4.fa.fai'
    GTFFN = '../datafiles/Anopheles-gambiae-PEST_BASEFEATURES_AgamP4.11.gtf'
elif GENOME == 'Aaegypti_L5.1':
    GENE_PREFIX = "AAEL"
    CHROMS = ['1','2','3']
    REF_FAI_FN = '/data/reference/AaegL5.fa.fai'
    GTFFN = "../datafiles/Aedes-aegypti-LVP_AGWG_BASEFEATURES_AaegL5.1.gtf"
else:
    assert False, "unknown GENOME '"+GENOME+"'"

# misc/testing
SUBSAMPLE_N = None
IGNORE_ONE_CALL_VARIANTS = False

## get chrom sizes and CDS info
`ref_fai` and `cdslist`

In [4]:
ref_fai = pd.read_csv(REF_FAI_FN, delimiter='\t', header=None, index_col=0,
                      names=['len','offset','line_bases', 'line_width'])

In [5]:
d = pd.read_csv(GTFFN, sep='\t', comment='#', header=None,
            names=['seqid',
                   'source',
                   'type',
                   'start',
                   'end',
                   'score',
                   'strand',
                   'phase',
                   'attributes'],
                dtype={'seqid':str})

# total number of genes
t = d.loc[d['type']=='gene' ,:].copy(deep=True)
t['gene_id'] = t['attributes'].apply(
                    lambda x: dict([_.strip().split() for _ in
                    x.split(';') if _])['gene_id'].strip('"'))
print("total number of genes in gtf:", t['gene_id'].unique().shape[0])

# list of CDS
d = d.loc[d['type']=='CDS' ,:]
d['gene_id'] = d['attributes'].apply(
                    lambda x: dict([_.strip().split() for _ in
                    x.split(';') if _])['gene_id'].strip('"'))
d['transcript_id'] = d['attributes'].apply(
                    lambda x: dict([_.strip().split() for _ in
                    x.split(';') if _])['transcript_id'].strip('"'))
# filter to only CRHOMS
d = d.loc[d['seqid'].isin(CHROMS) ,:]
cdslist = d

print('Total coding size (sum of CDS lens)',(d['end']-d['start']+1).sum())

total number of genes in gtf: 19712
Total coding size (sum of CDS lens) 59121227


## Load callset

In [6]:
callset = zarr.open_group(ZVCF, mode='r')
if CHROMS is None: # use all chroms?
    CHROMS = list(callset.keys())

## Load metadata

In [7]:
if SAMPLE_LIST_FN is None: # use all samples
    meta = pd.DataFrame(list(callset.values())[0]['samples'][:])
    meta['callset_idx'] = range(meta.shape[0])
else:
    # load the sample list and make meta dataframe
    if isinstance(SAMPLE_LIST_FN, str): # assume a simple list file
        meta = pd.read_csv(SAMPLE_LIST_FN, comment='#', header=None, index_col=0)
    else: # parse ag1000g samples.meta.txt with SAMPLE_LIST_FN really being [M|S,filename]
        meta = pd.read_csv(SAMPLE_LIST_FN[1], delimiter='\t', comment='#', index_col=0)
        meta = meta[meta['m_s']==SAMPLE_LIST_FN[0]]
    all_callset_samples = list(list(callset.values())[0]['samples'])
    meta['callset_idx'] = [all_callset_samples.index(x) for x in meta.index]
meta.index.name = 'sample'

# Optionally subsample to check sample size effects
if SUBSAMPLE_N:
    meta = meta.sample(n=SUBSAMPLE_N)

# ensure it is sorted by callset_idx (otherwise makes later steps terribly inefficient)
meta = meta.sort_values('callset_idx')

print("number of samples",meta.shape[0])
# meta

number of samples 132


# Compute nucleotide diversity and depth of coverage
over coding transcript loci

In [8]:
seqdiv = OrderedDict()
numloci = OrderedDict()
dpmedian = OrderedDict()
dpmean = OrderedDict()
dpstd = OrderedDict()
dpall = np.array([])

for chrom in CHROMS:
    print(chrom)

    #### build mask of CDS positions for chrom
    # (used as accessibility mask for calculatons)
    chrom_len = ref_fai.loc[chrom,'len']
    t = cdslist[cdslist['seqid']==chrom]
    acc_mask = np.full((chrom_len), False, dtype=bool)
    def _set_acc_mask(start, end):
        acc_mask[start-1:end] = True # note mask is 0-baesd half open positions
    _ = t.loc[:,['start','end']].apply(lambda x: _set_acc_mask(x['start'],x['end']), axis=1)

    numloci[chrom] = acc_mask.sum()
    
    # display(acc_mask[157497:]) # AgamP4.11 2L should have False,True,True,...
    # display(acc_mask[209814:]) # AgamP4.11 2L should have True,True,False,...

    #### Load and filter the callset data for this chrom
    ## Get the positions of variants on this chrom (possible subsetting samples)
    pos = allel.SortedIndex(callset[chrom]['variants/POS'])
    g = allel.GenotypeDaskArray(callset[chrom]['calldata/GT']).subset(None, meta['callset_idx'].values)
    n_variants_in = g.shape[0]
    
    # ensure number of samples is same
    assert meta.shape[0] == g.shape[1]
    
    # accessibility filter
    flt = pos.locate_ranges(t['start'], t['end'], strict=False)
    print("coding transcript filter passes: {} of {} = {:.3f}%".format(
        flt.sum(), flt.shape[0], 100*flt.sum()/flt.shape[0]))

    # @TCC TEMP/DEBUG minimal code to disable acc_mask
#     acc_mask = np.full((chrom_len), True, dtype=bool)
#     flt = np.full((pos.shape[0]), True, dtype=bool)
    
    # depth of coverage over flt (coding transcripts) loci
    d = callset[chrom]['calldata/DP']
    d = d.get_orthogonal_selection((np.flatnonzero(flt), meta['callset_idx'].values))
    d = d.clip(min=0) # replace all negative values with 0
    dpmedian[chrom] = np.median(d)
    dpmean[chrom] = d.flatten().mean()
    dpstd[chrom] = d.flatten().std()
    # @TCC TODO less memory apporach for median calcs histogram and adds them for each new chrom
    dpall = np.concatenate((dpall, d.flatten())) # this makes a huge list (samples*all_loci)
    del d
    
    # @TCC TODO should be able to speedup by applying filter to g here
    
    # get major allele frequencies
    ac = g.count_alleles().compute()
    # only variant alleles matter, so only take those
    flt_var = ac.is_variant()
    print('variant filter passes:', flt_var.sum())
    flt = (flt & flt_var)

    if IGNORE_ONE_CALL_VARIANTS:
        flt_one_call = ac[:,1:].sum(axis=1)>1
        print('ignore-one-call variant filter passes:', flt_one_call.sum())
        flt = (flt & flt_one_call)

    ac = ac.compress(flt, axis=0)
    vpos = pos.compress(flt, axis=0) # `[:]` to loads into memory

    n_variants_var = ac.shape[0]
    print("variants : {} of {} = {:.3f}%".format(n_variants_var, n_variants_in,
                                                     100*n_variants_var/n_variants_in))
    seqdiv[chrom] = allel.sequence_diversity(vpos, ac, is_accessible=acc_mask)
seqdiv

1
coding transcript filter passes: 306081 of 19363735 = 1.581%
variant filter passes: 19363735
variants : 306081 of 19363735 = 1.581%
2
coding transcript filter passes: 452676 of 33785440 = 1.340%
variant filter passes: 33785440
variants : 452676 of 33785440 = 1.340%
3
coding transcript filter passes: 390104 of 27317076 = 1.428%
variant filter passes: 27317076
variants : 390104 of 27317076 = 1.428%


OrderedDict([('1', 0.010523741627811483),
             ('2', 0.009221635796237162),
             ('3', 0.008789529658812079)])

In [9]:
display(numloci)

d = pd.DataFrame(seqdiv, index=[SETNAME])
# compute weighted (by number of accessible loci) average
d['overall'] = sum([d[k]*numloci[k] for k in numloci.keys()])/sum(numloci.values())
# print(d.to_string())
d

OrderedDict([('1', 5592763), ('2', 9206495), ('3', 8315366)])

,1,2,3,overall
VGL-Aaeg,0.010524,0.009222,0.00879,0.009381


In [10]:
dpdf = pd.DataFrame(dpmedian, index=[SETNAME+' median'])
dpdf.loc[SETNAME+' mean',:] = dpmean
dpdf.loc[SETNAME+' std',:] = dpstd

display(dpdf)
print(SETNAME, 'overall median converage (across acc loci):', np.median(dpall))

print('overall mean:', sum([dpmean[k]*numloci[k] for k in numloci.keys()])/sum(numloci.values()) )

,1,2,3
VGL-Aaeg median,10.000000,10.000000,10.000000
VGL-Aaeg mean,10.285799,10.514316,10.665598
VGL-Aaeg std,7.096819,6.669656,7.110822


VGL-Aaeg overall median converage (across acc loci): 10.0
overall mean: 10.513447638922914
